In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from functools import partial
from multiprocessing import Pool

import scarplet as sl
from scarplet.WindowedTemplate import Scarp

In [ ]:
data = sl.load('data/faultzone.tif')

In [ ]:
# Start separate search tasks
age = 10.
angles = np.linspace(-np.pi / 2, np.pi / 2, 181)
nprocs = 3
pool = mp.Pool(processes=nprocs)
wrapper = partial(sl.match_template, data, Scarp, scale, age)
results = pool.imap(wrapper, angles, chunksize=1)

In [ ]:
# Reduce the final results as they are completed
ny, nx = data.shape
best = sl.compare(results, nx, ny)

In [ ]:
sl.plot_results(data, best)